In [240]:
import numpy as np
import pandas as pd
import utils
from pymer4.models import Lmer, Lm
import ast

In [241]:
data = pd.read_csv("data/measures.csv")

condition_map = utils.get_nudge_condition_map()
data["condition"] = pd.Categorical(data["a_values"].map(condition_map), categories=condition_map.values(), ordered=True)
data["condition_coded"] = data["condition"].cat.codes

In [242]:
column_names = {"Estimate": "$b$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%"}
var_names = {"tta_0": "$\\textrm{TTA}_0$", "condition_coded": "condition", "tta_0:condition_coded": "$\\textrm{TTA}_0$:condition",
             "decisionStay:tta_0": "decision Stay:$\\textrm{TTA}_0$", "decisionStay:condition_coded": "decision Stay:condition"}

# Decision

In [236]:
model_is_go_decision = Lmer("is_go_decision ~ tta_0+condition_coded + (1 | subj_id) ", data=data, family="binomial")
model_is_go_decision_fit = model_is_go_decision.fit()
print(model_is_go_decision.coefs)

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: is_go_decision~tta_0+condition_coded+(1|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 2800	 Groups: {'subj_id': 14.0}

Log-likelihood: -1307.765 	 AIC: 2623.531

Random effects:

                Name   Var    Std
subj_id  (Intercept)  2.41  1.552

No random effect correlations specified

Fixed effects:

                 Estimate     2.5_ci   97.5_ci        SE        OR  OR_2.5_ci  \
(Intercept)     -9.909279 -11.268453 -8.550106  0.693469  0.000050   0.000013   
tta_0            1.816345   1.605998  2.026692  0.107322  6.149342   4.982831   
condition_coded  0.505252   0.432648  0.577855  0.037043  1.657403   1.541333   

                 OR_97.5_ci      Prob  Prob_2.5_ci  Prob_97.5_ci     Z-stat  \
(Intercept)        0.000194  0.000050     0.000013      0.000193 -14.289440   
tta_0              7.588941  0.860127     0.832855      0.883571  16.924280   
condition_coded    1.782212  0.623693     0.606506      0.640574  13.639449   

               

In [213]:
df = model_is_go_decision.coefs.loc[:, ["Estimate", "2.5_ci", "97.5_ci", "Z-stat", "P-val"]]

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('figures/tab_decision.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision", caption="Coefficients of the logistic regression describing the final decision"
     )
)

# Response time

In [215]:
model_RT = Lmer("RT ~ decision + tta_0 + condition_coded + tta_0:decision + condition_coded:decision + (1 | subj_id) ", data=data, family="gaussian")
model_RT_fit = model_RT.fit()

df = model_RT.coefs.loc[:, ["Estimate", "2.5_ci", "97.5_ci", "T-stat", "P-val"]]

print(df)

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


**NOTE**: Column for 'residuals' not created in model.data, but saved in model.resid only. This is because you have rows with NaNs in your data.

**NOTE** Column for 'fits' not created in model.data, but saved in model.fits only. This is because you have rows with NaNs in your data.

Formula: RT~decision+tta_0+condition_coded+tta_0:decision+condition_coded:decision+(1|subj_id)

Family: gaussian	 Inference: parametric

Number of observations: 2700	 Groups: {'subj_id': 14.0}

Log-likelihood: -2248.742 	 AIC: 4497.484

Random effects:

                 Name    Var    Std
subj_id   (Intercept)  0.063  0.251
Residual               0.345  0.587

No random effect correlations specified

Fixed effects:

                              Estimate    2.5_ci   97.5_ci    T-stat  \
(Intercept)                  -0.769750 -1.220119 -0.319381 -3.349879   
decisionStay                  1.540577  1.014194  2.066959  5.736273   
tta_0                         0.228416  0.147529  0.309304  5.534708   
conditi

In [216]:
styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('figures/tab_RT.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT", caption="Coefficients of the multilevel linear regression describing the response time"
     )
)

# Negative rating

In [217]:
model_rating = Lmer("is_negative_rating ~ decision + tta_0 + condition_coded + (1 | subj_id) ", data=data, family="binomial")
model_rating_fit = model_rating.fit()

df = model_rating.coefs.loc[:, ["Estimate", "2.5_ci", "97.5_ci", "Z-stat", "P-val"]]

print(df)

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: is_negative_rating~decision+tta_0+condition_coded+(1|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 2700	 Groups: {'subj_id': 14.0}

Log-likelihood: -451.232 	 AIC: 912.464

Random effects:

                Name    Var    Std
subj_id  (Intercept)  1.216  1.103

No random effect correlations specified

Fixed effects:

                  Estimate     2.5_ci   97.5_ci     Z-stat         P-val
(Intercept)     -10.871522 -13.248237 -8.494807  -8.965228  3.096406e-19
decisionStay      2.393874   1.859025  2.928723   8.772403  1.748934e-18
tta_0             0.539995   0.161391  0.918599   2.795455  5.182671e-03
condition_coded   1.237107   1.045834  1.428380  12.676556  7.976016e-37


In [218]:
styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('figures/tab_rating.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:rating", caption="Multilevel logistic regression describing the negative rating"
     )
)

# Posthoc testing

In [247]:
model_is_go_decision = Lmer("is_go_decision ~ tta_0 + condition + (1 | subj_id) ", data=data, family="binomial")
model_is_go_decision_fit = model_is_go_decision.fit(factors={"condition": list(condition_map.values())}, ordered=True, summarize=False)

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [249]:
marginal_estimates, comparisons = model_is_go_decision.post_hoc(
    marginal_vars="condition"#, grouping_vars="tta_0"
)

# "Cell" means of the ANOVA
print(marginal_estimates)

P-values adjusted by tukey method for family of 10 estimates
            condition  Estimate  2.5_ci  97.5_ci     SE   DF
0   Long acceleration    -1.011  -1.969   -0.053  0.489  inf
1  Acceleration nudge    -0.356  -1.313    0.600  0.488  inf
2      Constant speed    -0.243  -1.200    0.713  0.488  inf
3  Deceleration nudge     0.261  -0.695    1.218  0.488  inf
4   Long deceleration     1.234   0.275    2.193  0.489  inf


All subjects

In [250]:
print(comparisons)

                                  Contrast  Estimate  2.5_ci  97.5_ci     SE  \
0   Long acceleration - Acceleration nudge    -0.655  -1.050   -0.259  0.145   
1       Long acceleration - Constant speed    -0.768  -1.164   -0.372  0.145   
2   Long acceleration - Deceleration nudge    -1.272  -1.674   -0.870  0.147   
3    Long acceleration - Long deceleration    -2.245  -2.671   -1.819  0.156   
4      Acceleration nudge - Constant speed    -0.113  -0.504    0.277  0.143   
5  Acceleration nudge - Deceleration nudge    -0.617  -1.010   -0.224  0.144   
6   Acceleration nudge - Long deceleration    -1.590  -2.001   -1.180  0.151   
7      Constant speed - Deceleration nudge    -0.504  -0.896   -0.112  0.144   
8       Constant speed - Long deceleration    -1.477  -1.886   -1.068  0.150   
9   Deceleration nudge - Long deceleration    -0.973  -1.376   -0.570  0.148   

    DF  Z-stat  P-val  Sig  
0  inf  -4.517  0.000  ***  
1  inf  -5.287  0.000  ***  
2  inf  -8.626  0.000  ***  
3  

15 subjects

In [239]:
print(comparisons)

                                  Contrast  Estimate  2.5_ci  97.5_ci     SE  \
0   Long acceleration - Acceleration nudge    -0.542  -0.962   -0.121  0.154   
1       Long acceleration - Constant speed    -0.681  -1.102   -0.260  0.154   
2   Long acceleration - Deceleration nudge    -1.277  -1.706   -0.848  0.157   
3    Long acceleration - Long deceleration    -2.192  -2.650   -1.734  0.168   
4      Acceleration nudge - Constant speed    -0.139  -0.555    0.276  0.152   
5  Acceleration nudge - Deceleration nudge    -0.735  -1.156   -0.315  0.154   
6   Acceleration nudge - Long deceleration    -1.650  -2.096   -1.205  0.163   
7      Constant speed - Deceleration nudge    -0.596  -1.015   -0.177  0.154   
8       Constant speed - Long deceleration    -1.511  -1.954   -1.068  0.162   
9   Deceleration nudge - Long deceleration    -0.915  -1.353   -0.477  0.161   

    DF  Z-stat  P-val  Sig  
0  inf  -3.515  0.004   **  
1  inf  -4.413  0.000  ***  
2  inf  -8.118  0.000  ***  
3  